In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

from copy import copy
import datetime
import pickle
from geopy.geocoders import Nominatim

In [2]:
filename = "xgb_routeOptimization.sav"

In [3]:
loaded_model = pickle.load(open(filename, 'rb'))

[22:54:34] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.


In [4]:
#Sample date

date_list = [4, 6, 2016] #April 6, 2016

year = int(date_list[2])
month = int(date_list[1])
day = int(date_list[0])

my_date = datetime.date(year, month, day)

In [5]:
#Sample test locations
test_locations = {'L1': (40.819688, -73.915091),
                  'L2': (40.815421, -73.941761),
                  'L3': (40.764198, -73.910785),
                  'L4': (40.768790, -73.953285),
                  'L5': (40.734851, -73.952950),
                  'L6': (40.743613, -73.977998),
                  'L7': (40.745313, -73.993793),
                  'L8': (40.662713, -73.946101),
                  'L9': (40.703761, -73.886496),
                  'L10': (40.713620, -73.943076),
                  'L11': (40.725212, -73.809179)
             }

In [6]:
geolocator = Nominatim()
addresses = []

for key in test_locations:
    location = geolocator.reverse(test_locations[key])
    addresses.append(location.address)

/home/india/anaconda2/envs/pytorch_learning/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


In [7]:
addresses


['424, East 155th Street, Melrose, The Bronx, Bronx County, New York, 10455, United States of America',
 '137, West 136th Street, Harlem, Manhattan Community Board 10, Manhattan, New York County, New York, 10030, United States of America',
 '43-11, 28th Avenue, Steinway, Queens, Queens County, New York, 11103, United States of America',
 '435, East 74th Street, Upper East Side, Manhattan Community Board 8, Manhattan, New York County, New York, 10021, United States of America',
 '211, Freeman Street, Greenpoint, Brooklyn, Kings County, New York, 11222, United States of America',
 '232, East 32nd Street, Kips Bay, Manhattan Community Board 5, Manhattan, New York County, New York, 10016, United States of America',
 '159, West 25th Street, Penn South, Chelsea, Manhattan Community Board 4, Manhattan, New York County, New York, 10001, United States of America',
 '458, Lefferts Avenue, Crown Heights, Brooklyn, Kings County, New York, 11225, United States of America',
 '70-38, 67th Place, Quee

In [8]:
test_addresses = {'L1': '424 East 155th Street NY',
                  'L2': '137 West 136th Street NY',
                  'L3': '43-11 28th Avenue NY',
                  'L4': '435 East 74th Street NY',
                  'L5': '211 Freeman Street NY',
                  'L6': '232 East 32nd Street NY',
                  'L7': '159 West 25th Street NY',
                  'L8': '486 Brooklyn Avenue NY',
                  'L9': '70-38 67th Place NY',
                  'L10': '194 Devoe Street NY',
                  'L11': '158-46 76th Avenue NY'
             }

In [9]:
def create_guess(points):
    """
    Creates a possible path between all points, returning to the original.
    Input: List of point IDs
    """
    guess = copy(points)
    np.random.shuffle(guess)
    guess.append(guess[0])
    return list(guess)

create_guess(list(test_locations.keys()))

['L2', 'L6', 'L10', 'L1', 'L5', 'L8', 'L11', 'L7', 'L4', 'L3', 'L9', 'L2']

In [10]:
def create_generation(points, population=100):
    """
    Makes a list of guessed point orders given a list of point IDs.
    Input:
    points: list of point ids
    population: how many guesses to make
    """
    generation = [create_guess(points) for _ in range(population)]
    return generation

test_generation = create_generation(list(test_locations.keys()), population=10)
print(test_generation)

[['L9', 'L5', 'L3', 'L11', 'L6', 'L8', 'L4', 'L2', 'L7', 'L10', 'L1', 'L9'], ['L3', 'L11', 'L4', 'L9', 'L1', 'L2', 'L8', 'L5', 'L10', 'L7', 'L6', 'L3'], ['L10', 'L11', 'L5', 'L4', 'L1', 'L3', 'L2', 'L9', 'L7', 'L6', 'L8', 'L10'], ['L7', 'L5', 'L10', 'L8', 'L11', 'L1', 'L4', 'L3', 'L2', 'L9', 'L6', 'L7'], ['L11', 'L4', 'L7', 'L2', 'L9', 'L8', 'L5', 'L1', 'L6', 'L3', 'L10', 'L11'], ['L4', 'L5', 'L9', 'L3', 'L6', 'L11', 'L1', 'L7', 'L2', 'L10', 'L8', 'L4'], ['L8', 'L4', 'L2', 'L11', 'L7', 'L10', 'L5', 'L6', 'L3', 'L9', 'L1', 'L8'], ['L3', 'L4', 'L10', 'L9', 'L8', 'L1', 'L7', 'L5', 'L6', 'L2', 'L11', 'L3'], ['L10', 'L3', 'L8', 'L2', 'L11', 'L7', 'L9', 'L5', 'L1', 'L6', 'L4', 'L10'], ['L5', 'L9', 'L11', 'L7', 'L1', 'L3', 'L4', 'L2', 'L6', 'L8', 'L10', 'L5']]


In [11]:
def travel_time_between_points(point1_id, point2_id, hour, date, 
                               store_and_fwd_flag = 0, pickup_minute = 0):
    """
    Given two points, this calculates travel between them based on a XGBoost predictive model
    """
    
    model_data = {'pickup_longitude' : point1_id[1],
                  'pickup_latitude' : point1_id[0],
                  'dropoff_longitude' : point2_id[1],
                  'dropoff_latitude' : point2_id[0],
                  'latitude_difference' : point2_id[0] - point1_id[0],
                  'longitude_difference' : point2_id[1] - point1_id[1],
                  'distance' : 0.621371 * 6371 * (abs(2 * np.arctan2(np.sqrt(np.square(np.sin((abs(point2_id[0] - point1_id[0]) * np.pi / 180) / 2))), 
                                  np.sqrt(1-(np.square(np.sin((abs(point2_id[0] - point1_id[0]) * np.pi / 180) / 2)))))) + \
                                     abs(2 * np.arctan2(np.sqrt(np.square(np.sin((abs(point2_id[1] - point1_id[1]) * np.pi / 180) / 2))), 
                                  np.sqrt(1-(np.square(np.sin((abs(point2_id[1] - point1_id[1]) * np.pi / 180) / 2)))))))
                 }

    df = pd.DataFrame([model_data], columns=model_data.keys())
    
    pred = np.exp(loaded_model.predict(xgb.DMatrix(df))) - 1
    
    return pred[0]

In [12]:
coordinates = test_locations

In [ ]:
def fitness_score(guess):
    """
    Loops through the points in the guesses order and calculates
    how much distance the path would take to complete a loop.
    Lower is better.
    """
    score = 0
    for ix, point_id in enumerate(guess[:-1]):
        score += travel_time_between_points(coordinates[point_id], coordinates[guess[ix+1]], 11, my_date)
    return score

def check_fitness(guesses):
    """
    Goes through every guess and calculates the fitness score. 
    Returns a list of tuples: (guess, fitness_score)
    """
    fitness_indicator = []
    for guess in guesses:
        fitness_indicator.append((guess, fitness_score(guess)))
    return fitness_indicator

print(check_fitness(test_generation))